Method and tests from paper:

@article{mara2015non,
  title={Non-parametric methods for global sensitivity analysis of model output with dependent inputs},
  author={Mara, Thierry A and Tarantola, Stefano and Annoni, Paola},
  journal={Environmental modelling \& software},
  volume={72},
  pages={173--183},
  year={2015},
  publisher={Elsevier}
}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm

from SALib.analyze.sobol_corr import analyze

In [ ]:
from SALib.sample.sobol_corr import sample


def get_sensitivity_stats(problem, si_fabric, n=100, plotting=False):
    results = {}
    for _ in tqdm(range(n)):
        Sis = si_fabric(problem)
        for k, v in Sis.items():
            if k not in results:
                results[k] = [v]
            else:
                results[k].append(v)

    for k in results:
        results[k] = np.array(results[k])

    if plotting:
        for k in results:
            for i in range(problem["num_vars"]):
                print("===========")
                print(k, problem["names"][i])
                print("median: ", np.median(results[k][:, i]))
                print("mean: ", np.mean(results[k][:, i]))
                print("std: ", np.std(results[k][:, i]))
                plt.title(k)
                sns.distplot(results[k][:, i])
                plt.show()
                plt.close()

    return results

In [ ]:
def make_Si(problem):
    sample_args = {
        "n_sample": 1000,
    }

    analyze_args = {
        **sample_args,
        "n_boot": 100,
        "estimator": "soboleff1",
    }

    x = sample(problem, **sample_args)
    y = problem["func"](x)
    return analyze(problem, x, y, **analyze_args)

# Test 1

In [ ]:
def make_problem1():
    def func(v):
        return np.sum(v, axis=1)  # f(x) = x1+x2+x3

    def si_analytical():
        return {
            "S1_ind": [0.02, 0.05, 0.03],
            "ST_ind": [0.02, 0.05, 0.03],
            "S1_full": [0.95, 0.40, 0.60],
            "ST_full": [0.95, 0.40, 0.60],
        }

    problem = {
        "num_vars": 3,
        "names": ["x1", "x2", "x3"],
        "distrs": ["norm", "norm", "norm"],
        "bounds": [
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
        ],
        "corr": [
            [1.0, 0.5, 0.8],
            [0.5, 1.0, 0.0],
            [0.8, 0.0, 1.0],
        ],
        "func": func,
        "analytical": si_analytical,
    }

    return problem

In [ ]:
problem = make_problem1()
results_c = get_sensitivity_stats(problem, make_Si, n=500, plotting=False)
results_a = problem["analytical"]()

In [ ]:
print("Calculated: S1_ind", results_c["S1_ind"].mean(axis=0))
print("Analytical: S1_ind", results_a["S1_ind"], "\n")

print("Calculated: ST_ind", results_c["ST_ind"].mean(axis=0))
print("Analytical: ST_ind", results_a["ST_ind"], "\n")

print("Calculated: S1_full", results_c["S1_full"].mean(axis=0))
print("Analytical: S1_full", results_a["S1_full"], "\n")

print("Calculated: ST_full", results_c["ST_full"].mean(axis=0))
print("Analytical: ST_full", results_a["ST_full"])

# Test 2

In [ ]:
def make_problem2():
    def func(v):
        return np.sum(v, axis=1)  # f(x) = x1+x2+x3

    def si_analytical():
        return {
            "S1_ind": [0.70, 0.37, 0.50],
            "ST_ind": [0.70, 0.37, 0.50],
            "S1_full": [0.49, 0.05, 0.25],
            "ST_full": [0.49, 0.05, 0.25],
        }

    problem = {
        "num_vars": 3,
        "names": ["x1", "x2", "x3"],
        "distrs": ["norm", "norm", "norm"],
        "bounds": [
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
        ],
        "corr": [
            [1.0, -0.5, 0.2],
            [-0.5, 1.0, -0.7],
            [0.2, -0.7, 1.0],
        ],
        "func": func,
        "analytical": si_analytical,
    }

    return problem

In [ ]:
problem = make_problem2()
results_c = get_sensitivity_stats(problem, make_Si, n=500, plotting=False)
results_a = problem["analytical"]()

In [ ]:
print("Calculated: S1_ind", results_c["S1_ind"].mean(axis=0))
print("Analytical: S1_ind", results_a["S1_ind"], "\n")

print("Calculated: ST_ind", results_c["ST_ind"].mean(axis=0))
print("Analytical: ST_ind", results_a["ST_ind"], "\n")

print("Calculated: S1_full", results_c["S1_full"].mean(axis=0))
print("Analytical: S1_full", results_a["S1_full"], "\n")

print("Calculated: ST_full", results_c["ST_full"].mean(axis=0))
print("Analytical: ST_full", results_a["ST_full"])